In [2]:
import numpy as np
import pandas as pd
import kagglehub
import datetime as dt 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

Load Data

In [4]:
df = pd.read_csv('Donations_accepted_by_political_parties.csv', dtype={
    'index': 'int64',
    'ECRef' : 'string',
    'RegulatedEntityName': 'category',
    'RegulatedEntityType': 'category',
    'Value': 'string',
    "AcceptedDate": 'string',
    "AccountingUnitName": 'category',
    "DonorName": 'category',
    "AccountingUnitsAsCentralParty": 'bool',
    'IsSponsorship': 'bool',
    'DonorStatus': 'category',
    'RegulatedDoneeType': 'category',
    'CompanyRegistrationNumber': 'string',
    'Postcode': 'string',
    'DonationType': 'category',
    'NatureOfDonation': 'category',
    'PurposeOfVisit': 'string',
    'DonationAction': 'category',
    'ReceivedDate': 'string',
    'ReportedDate': 'string',
    'IsReportedPrePoll': 'string',
    'ReportingPeriodName': 'string',
    'IsBequest': 'bool',
    'IsAggregation': 'bool',
    'RegulatedEntityId': 'category',
    'AccountingUnitId': 'category',
    'DonorId': 'category',
    'CampaigningName': 'category',
    'RegisterName': 'string',
    'IsIrishSource': 'category'
    })

Correct Datatypes

In [62]:
df['Value'] = df['Value'].replace({'\£': '', ',': ''}, regex=True).astype(float)
df['ReceivedDate'] = pd.to_datetime(df['ReceivedDate'], format='%d/%m/%Y')
df['ReportedDate'] = pd.to_datetime(df['ReportedDate'], format='%d/%m/%Y')
df['AcceptedDate'] = pd.to_datetime(df['AcceptedDate'], format='%d/%m/%Y')

Review Data to identify Unnecessary Columns and Potential improved Index

In [75]:
summary_df = pd.DataFrame({
    'DataType': df.dtypes,
    'UniqueValues': df.nunique()
}).reset_index().rename(columns={'index': 'Column'})
print(summary_df)

                           Column        DataType  UniqueValues
0                           index           int64         65278
1                           ECRef          string         65278
2             RegulatedEntityName        category          1384
3             RegulatedEntityType        category             4
4                           Value         float64         11253
5                    AcceptedDate  datetime64[ns]          6265
6              AccountingUnitName        category          1856
7                       DonorName        category         19010
8   AccountingUnitsAsCentralParty            bool             2
9                   IsSponsorship            bool             2
10                    DonorStatus        category            13
11             RegulatedDoneeType        category            13
12      CompanyRegistrationNumber          string          3558
13                       Postcode          string          4559
14                   DonationType       

Change index to ECRef as evidently a unique identifier

In [7]:
df.set_index('ECRef')
df.describe()

,index,Value
count,65278.000000,6.527800e+04
mean,32638.500000,1.672815e+04
std,18844.279773,8.377591e+04
min,0.000000,0.000000e+00
25%,16319.250000,1.530000e+03
50%,32638.500000,3.000000e+03
75%,48957.750000,9.000000e+03
max,65277.000000,5.000000e+06


Review DonorStatus, DonationAction, DonationType, NatureOfDonation

In [82]:
df.groupby('DonorStatus').agg('count')

,index,ECRef,RegulatedEntityName,RegulatedEntityType,Value,AcceptedDate,AccountingUnitName,DonorName,AccountingUnitsAsCentralParty,IsSponsorship,...,IsAggregation,RegulatedEntityId,AccountingUnitId,DonorId,CampaigningName,RegisterName,IsIrishSource,Value_Category,DaysToNextElection,DaysToNextElection2
DonorStatus,,,,,,,,,,,,,,,,,,,,,
Building Society,3,3,3,3,3,3,0,3,3,3,...,3,3,0,3,0,3,3,3,3,3
Company,11883,11883,11883,11883,11883,11712,9756,11883,11883,11883,...,11883,11883,5272,11883,2,11558,11883,11883,11883,11883
Friendly Society,659,659,659,659,659,659,650,659,659,659,...,659,659,296,659,0,659,659,659,659,659
Impermissible Donor,254,254,254,254,254,2,220,254,254,254,...,254,254,64,254,0,248,254,254,254,254
Individual,29434,29434,29434,29434,29434,28977,26529,29430,29434,29434,...,29434,29434,16330,29434,3,28684,29434,29434,29434,29434
Limited Liability Partnership,477,477,477,477,477,475,424,477,477,477,...,477,477,104,477,0,471,477,477,477,477
Other,1757,1757,1757,1757,1757,1754,33,1757,1757,1757,...,1757,1757,20,1757,0,1744,1757,1757,1757,1757
Public Fund,1726,1726,1726,1726,1726,1720,1643,1726,1726,1726,...,1726,1726,289,1726,0,1713,1726,1726,1726,1726
Registered Political Party,1029,1029,1029,1029,1029,1029,918,1029,1029,1029,...,1029,1029,774,1029,0,1022,1029,1029,1029,1029


In [83]:
df.groupby('DonationAction').agg('count')

,index,ECRef,RegulatedEntityName,RegulatedEntityType,Value,AcceptedDate,AccountingUnitName,DonorName,AccountingUnitsAsCentralParty,IsSponsorship,...,IsAggregation,RegulatedEntityId,AccountingUnitId,DonorId,CampaigningName,RegisterName,IsIrishSource,Value_Category,DaysToNextElection,DaysToNextElection2
DonationAction,,,,,,,,,,,,,,,,,,,,,
Deferred,19,19,19,19,19,0,17,16,19,19,...,19,19,9,16,0,18,16,19,19,19
Forfeited,33,33,33,33,33,0,31,20,33,33,...,33,33,20,20,0,33,20,33,33,33
Returned,231,231,231,231,231,0,202,220,231,231,...,231,231,48,224,0,226,224,231,231,231


In [84]:
df.groupby('DonationType').agg('count')

,index,ECRef,RegulatedEntityName,RegulatedEntityType,Value,AcceptedDate,AccountingUnitName,DonorName,AccountingUnitsAsCentralParty,IsSponsorship,...,IsAggregation,RegulatedEntityId,AccountingUnitId,DonorId,CampaigningName,RegisterName,IsIrishSource,Value_Category,DaysToNextElection,DaysToNextElection2
DonationType,,,,,,,,,,,,,,,,,,,,,
Cash,51642,51642,51642,51642,51642,51008,47581,51642,51642,51642,...,51642,51642,31333,51642,5,50561,51642,51642,51642,51642
Exempt Trust,469,469,469,469,469,468,467,469,469,469,...,469,469,432,469,0,467,0,469,469,469
Non Cash,8308,8308,8308,8308,8308,8271,6993,8308,8308,8308,...,8308,8308,4214,8308,0,8229,8308,8308,8308,8308
Permissible Donor Exempt Trust,171,171,171,171,171,171,10,171,171,171,...,171,171,10,171,0,171,0,171,171,171
Public Funds,1713,1713,1713,1713,1713,1713,1638,1713,1713,1713,...,1713,1713,289,1713,0,1708,1713,1713,1713,1713
Visit,2634,2634,2634,2634,2634,2634,18,2634,2634,2634,...,2634,2634,0,2634,0,2634,2634,2634,2634,2634
Impermissible Donor,260,260,260,260,260,0,228,256,260,260,...,260,260,67,260,0,254,260,260,260,260
Total value of donations not reported individually,58,58,58,58,58,0,0,0,58,58,...,58,58,0,0,0,0,0,58,58,58
Unidentified Donor,23,23,23,23,23,0,22,0,23,23,...,23,23,10,0,0,23,0,23,23,23


In [85]:
df.groupby('NatureOfDonation').agg('count')

,index,ECRef,RegulatedEntityName,RegulatedEntityType,Value,AcceptedDate,AccountingUnitName,DonorName,AccountingUnitsAsCentralParty,IsSponsorship,...,IsAggregation,RegulatedEntityId,AccountingUnitId,DonorId,CampaigningName,RegisterName,IsIrishSource,Value_Category,DaysToNextElection,DaysToNextElection2
NatureOfDonation,,,,,,,,,,,,,,,,,,,,,
Administration services,456,456,456,456,456,454,378,456,456,456,...,456,456,282,456,0,452,456,456,456,456
Advertising,1168,1168,1168,1168,1168,1158,1069,1168,1168,1168,...,1168,1168,663,1168,0,1155,1163,1168,1168,1168
Assistance for Parties (Scottish Parliament),311,311,311,311,311,311,288,311,311,311,...,311,311,69,311,0,311,311,311,311,311
Auction prizes,628,628,628,628,628,626,608,628,628,628,...,628,628,220,628,0,628,628,628,628,628
Consultancy services,377,377,377,377,377,375,292,377,377,377,...,377,377,116,377,0,371,377,377,377,377
Cranborne Money (House of Lords),162,162,162,162,162,162,140,162,162,162,...,162,162,34,162,0,162,162,162,162,162
Financial Assistance for Parties in NI Assembly,259,259,259,259,259,259,259,259,259,259,...,259,259,12,259,0,259,259,259,259,259
Hospitality,184,184,184,184,184,184,133,184,184,184,...,184,184,59,184,0,184,184,184,184,184
Loan conversion,39,39,39,39,39,39,39,39,39,39,...,39,39,7,39,0,39,39,39,39,39


Remove the 
"Impermissible Donor" DonorStatus
"Forfeited" and "Returned" DonationActions.
"Impermissible Donor" DonationType

Copy removed records to "ExcludedData" dataframe, then create CleanedData Dataframe excluding the identified records

In [102]:
#Excludeddata = df[(df['DonorStatus']=="Impermissible Donor") | (df['DonationActions']==('Forfeited', 'Returned')) | (df[Donationtype]=='Impermissible Donor')]

Excludeddata = df[(df['DonorStatus']=="Impermissible Donor") | (df['DonationAction']==('Forfeited' , 'Returned'))| (df['DonationType']=='Impermissible Donor')]
Excludeddata

,index,ECRef,RegulatedEntityName,RegulatedEntityType,Value,AcceptedDate,AccountingUnitName,DonorName,AccountingUnitsAsCentralParty,IsSponsorship,...,IsAggregation,RegulatedEntityId,AccountingUnitId,DonorId,CampaigningName,RegisterName,IsIrishSource,Value_Category,DaysToNextElection,DaysToNextElection2
13588,13588,V0221129,Sharon Hodgson MP,Regulated Donee,3400.0,2015-09-18,NaN,Taipei Representative Office in the UK,False,False,...,False,1463,NaN,72314.0,NaN,Great Britain,False,4000.0,0,0
20338,20338,C0196962,Mr Mark Pritchard MP,Regulated Donee,1000.0,2014-07-09,NaN,Mr Frank Kaszynski,False,False,...,False,1514,NaN,68122.0,NaN,Great Britain,False,1000.0,391,391
64274,64274,I0033960,Liberal Democrats,Political Party,6380.0,NaT,Central Party,Miss R E Finey,False,False,...,False,90,NaN,33490.0,NaN,Great Britain,False,7500.0,1765,1765
64275,64275,I0052775,Liberal Democrats,Political Party,1000.0,NaT,Central Party,NaN,False,False,...,False,90,NaN,37474.0,NaN,Great Britain,False,1000.0,1430,1430
64276,64276,I0034040,Conservative and Unionist Party,Political Party,500.0,NaT,Central Party,Mr Matthew Peck,False,False,...,False,52,NaN,33570.0,NaN,Great Britain,False,1000.0,221,221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65262,65262,I0144765,Conservative and Unionist Party,Political Party,1000.0,NaT,Bristol North West,Mr John K Lewis,False,False,...,False,52,1009.0,54037.0,NaN,Great Britain,False,1000.0,200,200
65263,65263,I0034037,UK Independence Party (UKIP),Political Party,1000.0,NaT,Central Party,Mr John Haynes,False,False,...,False,85,NaN,33567.0,NaN,Great Britain,False,1000.0,160,160
65269,65269,I0033983,Conservative and Unionist Party,Political Party,5900.0,NaT,Witney,Mr Roger Fletcher,False,False,...,False,52,1724.0,33513.0,NaN,Great Britain,False,7500.0,1019,1019
65270,65270,I0088673,Liberal Democrats,Political Party,1000.0,NaT,Winchester District,St Cross Consulting Limited,False,False,...,False,90,2390.0,45945.0,NaN,Great Britain,False,1000.0,935,935


In [ ]:
CleanedData = 

In [8]:
df.groupby('DonationType').Value.agg(['min', 'max', 'sum', 'count', 'mean', 'median'])

,min,max,sum,count,mean,median
DonationType,,,,,,
Cash,0.00,5000000.00,8.411877e+08,51642,16288.828388,3000.000
Exempt Trust,350.00,330272.00,3.521248e+06,469,7507.992154,2500.000
Non Cash,10.96,1952374.76,7.373360e+07,8308,8875.012406,2678.505
Permissible Donor Exempt Trust,593.00,63000.00,1.334526e+06,171,7804.245673,2000.000
Public Funds,250.00,3685124.97,1.616066e+08,1713,94341.245470,13163.100
Visit,0.00,47252.72,9.104412e+06,2634,3456.496484,2854.210
Impermissible Donor,50.00,100000.00,7.998591e+05,260,3076.381231,1000.000
Total value of donations not reported individually,0.00,49085.00,6.425717e+05,58,11078.821897,6958.900
Unidentified Donor,300.00,10000.00,4.945000e+04,23,2150.000000,1000.000


Going to try to create a ML model that predicts the size of a donation based on the donor, donee, 

Step one is to create a grouping based on the Value donated.

In [32]:
maxvalue = df.Value.max()
print(maxvalue)

5000000.0


In [36]:
valueCategory = [0, 1000, 1500, 2000, 2500, 3000, 4000, 5000, 7500, 10000, 25000, 50000, 75000, 100000, 250000, 500000, 750000, 1000000, 2000000]

maxvalue = df.Value.max()

def categorize(total):
    if np.isnan(total):
        return None
    else: 
        for VC in valueCategory:        
            if total <= VC:
                return VC
            else:
                pass
        return maxvalue

df['Value_Category'] = df.Value.apply(categorize)

In [40]:
value_summary_df = df.groupby(['Value_Category']).agg({'Value': ['sum', 'count']})
value_summary_df

Value       
                         sum  count
Value_Category                     
0.0             0.000000e+00      6
1000.0          5.811508e+06   8658
1500.0          9.261251e+06   7128
2000.0          1.903649e+07  10204
2500.0          1.061907e+07   4489
3000.0          1.070453e+07   3708
4000.0          1.255565e+07   3463
5000.0          2.633579e+07   5415
7500.0          2.497021e+07   4078
10000.0         5.523923e+07   5881
25000.0         1.086985e+08   6319
50000.0         1.189506e+08   2870
75000.0         4.808200e+07    785
100000.0        6.224243e+07    659
250000.0        1.523126e+08    898
500000.0        1.704321e+08    477
750000.0        5.795796e+07     93
1000000.0       5.258238e+07     58
2000000.0       1.048143e+08     75
5000000.0       4.137328e+07     14

In [57]:
ElectionDates = ['2001/06/07 00:00:00','2005/05/05 00:00:00','2010/06/05 00:00:00', '2015/07/05 00:00:00', '2017/07/05 00:00:00', '2019/12/12 00:00:00', '2024/07/04 00:00:00']

def GenElectionRelation(R_Date):
    for ED in ElectionDates:
        R_Date2 = R_Date
        #R_Date2 = dt.datetime.strptime(R_Date, '%Y/%m/%d %H:%M:%S')
        ED2 = dt.datetime.strptime(ED, '%Y/%m/%d %H:%M:%S')
        if R_Date2 <= ED2:
            DaysDiff  = (ED2 - R_Date2)
            delta = DaysDiff.days
            return delta
        else:
            pass
    return 0

df['DaysToNextElection'] = df.ReceivedDate.apply(GenElectionRelation)


In [58]:
Limited_Columns_df=df[['RegulatedEntityType','Value','Value_Category','ReceivedDate','DaysToNextElection']]
Limited_Columns_df

,RegulatedEntityType,Value,Value_Category,ReceivedDate,DaysToNextElection
0,Regulated Donee,1876.56,2000.0,2019-09-02,101
1,Regulated Donee,2700.00,3000.0,2019-09-02,101
2,Regulated Donee,4852.58,5000.0,2019-08-30,104
3,Regulated Donee,2500.00,2500.0,2019-08-30,104
4,Regulated Donee,4524.05,5000.0,2019-08-29,105
...,...,...,...,...,...
65273,Permitted Participant,10000.00,10000.0,2016-04-04,457
65274,Permitted Participant,100000.00,100000.0,2016-02-08,513
65275,Permitted Participant,40000.00,50000.0,2016-06-17,383
65276,Political Party,50000.00,50000.0,2017-05-25,41


In [ ]:


df.corr()

,index,Value,AccountingUnitsAsCentralParty,IsSponsorship,IsBequest,IsAggregation,Value_Category,DaysToNextElection,DaysToNextElection2
index,1.000000,-0.011388,0.011167,-0.025772,-0.020355,0.078560,-0.010719,0.288154,0.288154
Value,-0.011388,1.000000,-0.032147,-0.011967,0.034117,-0.101568,0.968624,0.002698,0.002698
AccountingUnitsAsCentralParty,0.011167,-0.032147,1.000000,0.011080,-0.020012,0.299099,-0.029632,0.011320,0.011320
IsSponsorship,-0.025772,-0.011967,0.011080,1.000000,-0.011866,0.020286,-0.010826,0.027794,0.027794
IsBequest,-0.020355,0.034117,-0.020012,-0.011866,1.000000,-0.052486,0.031587,0.024464,0.024464
IsAggregation,0.078560,-0.101568,0.299099,0.020286,-0.052486,1.000000,-0.094245,0.071893,0.071893
Value_Category,-0.010719,0.968624,-0.029632,-0.010826,0.031587,-0.094245,1.000000,0.005939,0.005939
DaysToNextElection,0.288154,0.002698,0.011320,0.027794,0.024464,0.071893,0.005939,1.000000,1.000000
DaysToNextElection2,0.288154,0.002698,0.011320,0.027794,0.024464,0.071893,0.005939,1.000000,1.000000
